In [14]:
# You only need to run this once per machine
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U datasets scipy ipywidgets matplotlib

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [15]:
print("Hello")

Hello


In [17]:
# !pip install transformers torch accelerate bitsandbytes

In [18]:
# !pip install huggingface_hub

In [19]:
from huggingface_hub import login

import os
token = os.getenv("HUGGINGFACE_TOKEN")


login(token=access_token)

In [20]:
# =========================================
# 1) Imports
# =========================================
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments

In [21]:
from datasets import Dataset

In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments

# Load the dataset
csv_path = "/home/cse/Desktop/VOIP_VISHING/REPOS/conversation-prediction/FINAL_DATASET2.csv"
df = pd.read_csv(csv_path)

# Group by CONVERSATION_ID and create full conversations
df['FULL_CONVERSATION'] = df.groupby(['CONVERSATION_ID'])['TEXT'].transform(lambda x: ' '.join(x))

# Create partial conversations (excluding the last step)
df['PARTIAL_CONVERSATION'] = df.groupby(['CONVERSATION_ID'])['TEXT'].transform(lambda x: ' '.join(x[:-1]))

# Split the dataset into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1")

# Tokenize the data
def tokenize_function(examples):
    return tokenizer(examples['PARTIAL_CONVERSATION'], padding="max_length", truncation=True, max_length=512)

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_val = val_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/870 [00:00<?, ? examples/s]

Map:   0%|          | 0/97 [00:00<?, ? examples/s]

In [29]:
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments

# Load the model
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")


config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [30]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the model and tokenizer
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto")

tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [39]:
# Move the model to GPU
model = model.to("cuda")

# Test inference
input_text = "Hello?"
inputs = tokenizer(input_text, return_tensors="pt").to("cuda")

# Generate output
outputs = model.generate(**inputs, max_length=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Hello? I'm a student, and I'm trying to understand this problem about a function f


In [42]:
import torch
torch.cuda.empty_cache()

In [44]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,  # Reduced batch size
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,  # Gradient accumulation
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    save_steps=10_000,
    logging_dir='./logs',
    logging_steps=200,
    fp16=True,  # Mixed precision training
)


# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,  # Your tokenized training dataset
    eval_dataset=tokenized_val,     # Your tokenized validation dataset
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model
model.save_pretrained("./fine_tuned_deepseek_distill")
tokenizer.save_pretrained("./fine_tuned_deepseek_distill")

/home/cse/anaconda3/envs/voipvishing/lib/python3.13/site-packages/transformers/training_args.py:1612: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 594.00 MiB. GPU 0 has a total capacity of 15.69 GiB of which 288.62 MiB is free. Including non-PyTorch memory, this process has 15.35 GiB memory in use. Of the allocated memory 14.44 GiB is allocated by PyTorch, and 668.73 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)